In [90]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import DBSCAN, KMeans
from sklearn.manifold import TSNE
import re

#### Считываем данные из csv файла и удаляем данные, которые будут мешать дальнейшей кластеризации (Данные с ответом 4XX т.к. это ошибки и бесполезные символы)

In [91]:
df = pd.read_csv('allout.csv')
df = df.loc[df['meta3'] < 400.0]
df = df.dropna()
df['vector'] = df['vector'].astype('string')
df['vector'] = df['vector'].map(lambda x: re.sub('[^A-Za-z0-9:\/\"\':;=]+', '', x))
df['vector'].replace('', np.nan, inplace=True)
df = df.dropna()
df.shape

(5520, 7)

In [92]:
# Вектора документов
samples =df['vector'].tolist()
vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(1, 3))
vectors = vectorizer.fit_transform(samples)
# Кластеризация при помощи метода k-средних
res = KMeans(n_clusters=2, random_state=0).fit_predict(vectors)
set(res)

{0, 1}

### Визуализация при помощи TSNE

In [93]:
import plotly.express as px

res_df = pd.DataFrame(data=res, columns=['label'])
# Вектора для визуализации
X_embedded = TSNE(n_components=2).fit_transform(vectors)
res_df["x_component"]=X_embedded[:,0]
res_df["y_component"]=X_embedded[:,1]
    
fig = px.scatter(res_df, x="x_component", y="y_component", color = 'label', size_max=60)
fig.update_layout(height=800)
fig.show()

/usr/local/lib/python3.8/dist-packages/sklearn/manifold/_t_sne.py:780: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/usr/local/lib/python3.8/dist-packages/sklearn/manifold/_t_sne.py:790: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



### Вывод и анализ 2-х полученных кластеров

In [94]:
res_df = pd.DataFrame(np.concatenate((res_df.to_numpy(), df.to_numpy()), axis=1))

cluster1 = res_df.loc[res_df[0] == 0]
cluster2 = res_df.loc[res_df[0] == 1]

### Документы кластера first_cluster схожи с обычными (безопасными) запросами:

In [95]:
cluster1

,0,1,2,3,4,5,6,7,8,9
3,0.0,-19.960636,10.072229,7,7,950ebbb818650193f9328d75453e69cb,REQUEST_HEADERS,Referer,200.0,file:///Applications/Install/639E2536C2764671B...
4,0.0,-19.970129,10.071376,9,9,d2734f4ab17d4c4cd638f52a8628402a,REQUEST_HEADERS,Referer,200.0,file:///Applications/Install/65621A5D0DDE45B1A...
5,0.0,-42.821945,15.328206,11,11,a82230c545dcbc51e830255835ae25d6,REQUEST_ARGS,file,200.0,///images/stories/1/t1jpg
9,0.0,11.181169,-50.926765,23,23,ca33d909a87e1d1eea8791866ed106fb,REQUEST_COOKIES,anj,200.0,dTM7kM4/C/9P:57Xwb7NPL6xPSAeE=hrgJ7vjwyY2INN9:...
10,0.0,-46.644722,-3.956263,24,24,0bce3849ead8521394088d6599bf468b,REQUEST_ARGS,pll_language,200.0,file:///windows/winini
...,...,...,...,...,...,...,...,...,...,...
5500,0.0,-34.478355,-7.565727,974,967,ad65b3b71aa832e10fcc2aad2e18ad1b,REQUEST_PATH,REQUEST_PATH,200.0,/documents/nationalplan
5505,0.0,13.021069,-48.348137,981,974,1ba07ba33d164789e5547683be31a44d,REQUEST_COOKIES,anj,200.0,dTM7kM4/KUy5eyf/Eo/KEzIaUcuNQki0uf5Ybh23TSiFUX...
5506,0.0,-32.789646,11.714208,982,975,f8b7094163e78401f59a222d8c8a970c,REQUEST_PATH,REQUEST_PATH,200.0,/updates/kdb/i386/diffs/base155ekdc0mu
5509,0.0,-3.72049,-26.770832,985,978,6e9f96f4dc671a45d907589446dff6dc,REQUEST_GET_ARGS,dest,302.0,http://chechaqufulbacom/documentsrl=40515


### В случае же кластера second_cluster исходя из выборки, можно заметить подозрительный контент в запросах

In [96]:
cluster2

,0,1,2,3,4,5,6,7,8,9
0,1.0,-31.59137,59.051872,1,1,876c76e2173eabf5849c01d8f85de9a9,REQUEST_ARGS,action,304.0,dlattach;topic=41020;attach=7045;image
1,1.0,2.871564,-34.436966,5,5,d3743773c596f478093bcc8c2933db65,REQUEST_ARGS_KEYS,M}PÂ¶WMÂ¤_Â§ÃÃ <Ã¬?{#Â8Ã»ÎA?tÂ5ÂÂ©Ø½...,200.0,MPWM8At5
2,1.0,-4.430802,39.482944,6,6,a43d37066673e80c8b59826f731fbc4c,REQUEST_ARGS,image,200.0,/imgbasa/17896173683jpg
6,1.0,-14.924122,19.682026,13,13,f00d514a360a9926f1a54f99a9e7a942,REQUEST_PATH,REQUEST_PATH,200.0,/mail/spb/vkvasniknsf/Inbox/B481571280728846C1...
7,1.0,17.04377,10.223716,15,15,b7caea0cc6d15508ff30f95f3e599160,RESPONSE_HEADERS,RESPONSE_HEADERS.Age,200.0,1443
...,...,...,...,...,...,...,...,...,...,...
5514,1.0,52.168011,-40.921139,997,990,c07dc8984b8def157ef49ee21f31fde5,REQUEST_XML,REQUEST_XML./s:Envelope/s:Body/DeviceMetadataB...,302.0,DOID:HIDVID2717PIDF006REV0318
5516,1.0,20.206011,9.810391,1000,993,2f0f2a89b9742042d11bee029fdb9d3b,REQUEST_HEADERS,REQUEST_HEADERS.X-Forwarded-For,200.0,91185899
5517,1.0,0.004553,59.650383,1001,994,176ad9ac69bb3e8379ef6ebe17274165,REQUEST_GET_ARGS,ELEMENT_ID,200.0,33635'UNIONALLSELECTNULLNULLNULLCONCAT0x716a70...
5518,1.0,-0.874271,59.824852,1003,996,8c881888c7512fe84d5258c80f580376,REQUEST_GET_ARGS,ELEMENT_ID,200.0,33635UNIONALLSELECTNULLNULLNULLNULLNULLNULLNUL...


### По аналогии с предыдущими шагами разобьём second_cluster при помощи DBSCAN на несколько кластеров для анализа:

In [97]:
import plotly.express as px

# Вектора для визуализации
samples = cluster2[9].tolist()
vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(1, 3))
vectors = vectorizer.fit_transform(samples)
# Кластеризация при помощи DBSCAN
res = DBSCAN(eps=1.2, min_samples=30).fit_predict(vectors)
set(res)
# Визуализация при помощи TSNE
res_df = pd.DataFrame(data=res, columns=['label'])
# Применение алгоритма TSNE для получения двумерного представления tf-idf векторов для дальнейшей визуализации
X_embedded = TSNE(n_components=2).fit_transform(vectors)
res_df["x_component"]=X_embedded[:,0]
res_df["y_component"]=X_embedded[:,1]


fig = px.scatter(res_df, x="x_component", y="y_component", color = 'label', size_max=60)
fig.update_layout(height=800)
fig.show()

/usr/local/lib/python3.8/dist-packages/sklearn/manifold/_t_sne.py:780: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/usr/local/lib/python3.8/dist-packages/sklearn/manifold/_t_sne.py:790: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



# Анализ кластеров

In [98]:
res_df = np.concatenate((res_df.to_numpy(), cluster2.to_numpy()), axis=1)
res_df = pd.DataFrame(res_df)

### 1 кластер содержит в себе запросы с обычными http-заголовки, которые не являются вредоносными

In [99]:
# Первый кластер
res_df.loc[res_df[0] == 0]

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.0,17.353291,62.327984,1.0,-31.59137,59.051872,1,1,876c76e2173eabf5849c01d8f85de9a9,REQUEST_ARGS,action,304.0,dlattach;topic=41020;attach=7045;image
12,0.0,-13.722494,43.698895,1.0,-41.829285,40.995407,41,41,04370f50fc8b4f004dfef0fb3a291d16,REQUEST_ARGS,area,200.0,showposts;u=1377;PHPSESSID=cdf65ffa1ab9dc964ec...
22,0.0,4.611706,74.504509,1.0,-45.048885,63.218048,78,78,b8368fd449c9cd3026a6d240fe0b30ca,REQUEST_ARGS,action,200.0,dlattach;attach=1674;type=avatar
47,0.0,-22.877314,48.57056,1.0,-45.003426,48.993816,143,143,d76c5078969c6afc5ead717a4407869d,REQUEST_ARGS,action,200.0,profile;u=600;area=summary
74,0.0,-4.985611,44.081905,1.0,-29.356256,38.850822,228,227,ff0c1e4ef34b1230729fd85907974b33,REQUEST_ARGS,action,200.0,xml;limit=10;board=25;type=rss
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3219,0.0,-15.379148,43.86834,1.0,-42.747284,42.302052,756,750,6d646e528844cf9bcdfd63f3085d269d,REQUEST_ARGS,action,200.0,profile;area=showposts;sa=messages;u=598581
3237,0.0,5.998633,54.838104,1.0,-18.287354,43.568794,800,794,61d2d74249cfa2e2a124e24ab9a90640,REQUEST_ARGS,action,200.0,reporttm;topic=41519;msg=7085
3256,0.0,4.117302,49.097809,1.0,-20.523504,38.647285,854,848,d0fed71803586a137f56a86f314ca5c3,REQUEST_ARGS,action,200.0,post;quote=122299;topic=795075;lastmsg=123898
3261,0.0,-15.199014,60.691307,1.0,-51.517681,35.766335,876,870,2716cac8bdb0f9e8301439ce66f60db0,REQUEST_ARGS,action,200.0,mgallery;sa=item;id=2566sort=0;asc


### 2 кластер содержит в себе документы с потенциально опасным контентом. Можно увидеть sql и js инъекции в теле запросов

In [100]:
# 2 кластер
res_df.loc[res_df[0] == 1]

,0,1,2,3,4,5,6,7,8,9,10,11,12
3,1.0,39.044487,13.370341,1.0,-14.924122,19.682026,13,13,f00d514a360a9926f1a54f99a9e7a942,REQUEST_PATH,REQUEST_PATH,200.0,/mail/spb/vkvasniknsf/Inbox/B481571280728846C1...
6,1.0,-82.702164,6.324742,1.0,-6.748511,94.347046,26,26,56616167ae6cff7829b5d8a2679e7af9,REQUEST_ARGS,id,200.0,11printchr492bchr552bchr732bchr532bchr512bchr4...
9,1.0,34.136887,36.656925,1.0,25.257498,62.336678,34,34,daf4bede8e373833a3d0b01b5936d595,REQUEST_ARGS,action,200.0,verificationcode;vid=pm;rand=1ca59ab70accac76b...
10,1.0,-81.522827,6.290463,1.0,-6.50181,94.695297,35,35,74061910e04d75450cd6fc5a41f9380f,REQUEST_ARGS,products_id,200.0,65printchr492bchr552bchr732bchr532bchr512bchr4...
11,1.0,-61.014606,13.315672,1.0,-2.038282,73.450798,40,40,dbc09a238a5d6065f7dd6b94ec970f18,REQUEST_ARGS,gid,200.0,907printchr49chr55chr73chr53chr51chr48chr86chr...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3300,1.0,-45.569836,-38.919395,1.0,58.882927,-16.506325,992,985,05d9497f40eb98e3fa69065391e087e8,REQUEST_ARGS,id,200.0,236lsla;factor228000
3302,1.0,-24.678766,29.166245,1.0,52.168011,-40.921139,997,990,c07dc8984b8def157ef49ee21f31fde5,REQUEST_XML,REQUEST_XML./s:Envelope/s:Body/DeviceMetadataB...,302.0,DOID:HIDVID2717PIDF006REV0318
3304,1.0,-57.140808,-2.385825,1.0,0.004553,59.650383,1001,994,176ad9ac69bb3e8379ef6ebe17274165,REQUEST_GET_ARGS,ELEMENT_ID,200.0,33635'UNIONALLSELECTNULLNULLNULLCONCAT0x716a70...
3305,1.0,-57.892178,-1.91533,1.0,-0.874271,59.824852,1003,996,8c881888c7512fe84d5258c80f580376,REQUEST_GET_ARGS,ELEMENT_ID,200.0,33635UNIONALLSELECTNULLNULLNULLNULLNULLNULLNUL...


### В 3 кластере можно заметить обычные хэдеры http-запросов, что не является вредоносным контентом

In [101]:
# 3 кластер
res_df.loc[res_df[0] == 2]

,0,1,2,3,4,5,6,7,8,9,10,11,12
19,2.0,-23.881624,-51.921185,1.0,59.080036,4.360004,65,65,eb5b276fa48beb3b5772a0572cff48ad,RESPONSE_HEADERS,RESPONSE_HEADERS.Date,200.0,Mon29Jan201811:29:23GMT
98,2.0,-20.029581,-51.107628,1.0,60.106983,8.829669,337,334,f2cb282cfed53808a90f0b037c6e744c,REQUEST_HEADERS,REQUEST_HEADERS.If-Modified-Since,200.0,Mon05Feb201802:53:25GMT
125,2.0,-19.866287,-49.369022,1.0,58.315689,8.904818,431,428,cd04ad17513334aaea4920b86855872a,RESPONSE_HEADERS,RESPONSE_HEADERS.Date,200.0,Thu01Feb201815:31:00GMT
501,2.0,-17.673813,-48.695786,1.0,56.396259,8.510484,669,668,3dbca410b90170ce8453f20ddd18bf45,REQUEST_COOKIES,REQUEST_COOKIES._LAST_VISIT,200.0,0602201810:26:25
518,2.0,-22.755577,-53.673676,1.0,61.317963,6.065875,725,724,07355f30f488d83c1dd6597a6ff29935,RESPONSE_HEADERS,RESPONSE_HEADERS.Date,200.0,Tue30Jan201816:41:18GMT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3071,2.0,-25.408581,-45.062992,1.0,54.426136,2.434824,320,318,99442391fcf8aa83aaac1dabdb463780,REQUEST_POST_ARGS,REQUEST_POST_ARGS.time0,200.0,20:32
3077,2.0,-18.836617,-52.253551,1.0,59.747837,10.153642,339,337,166cbfa7e9df014d2ccb5c9c0ec37744,REQUEST_HEADERS,REQUEST_HEADERS.If-Modified-Since,200.0,Sun04Feb201819:13:13GMT
3168,2.0,-22.258904,-51.372375,1.0,59.895615,4.978468,605,599,6056c2a73036d851ce4d87d0df11ee09,RESPONSE_HEADERS,RESPONSE_HEADERS.Date,200.0,Sun28Jan201815:01:56GMT
3222,2.0,-22.624054,-53.187744,1.0,60.902874,5.957878,765,759,e830979871e85ed4762ee11776fc9e0d,RESPONSE_HEADERS,RESPONSE_HEADERS.Date,200.0,Tue30Jan201811:49:34GMT


### В 4 кластере содержатся, насколько можно предположить, попытки аутентификации, что может свидетельствовать о брутфорсе, либо об обычных аутентификациях пользователей

In [102]:
# 4 кластер
res_df.loc[res_df[0] == 3]

,0,1,2,3,4,5,6,7,8,9,10,11,12
87,3.0,-38.825657,52.130096,1.0,75.692497,19.095062,285,284,1ba1331f31b84b6875134f847d08b28c,REQUEST_ARGS,action,302.0,login2;sa=check;member=1629456
202,3.0,-38.509781,50.924492,1.0,76.085724,18.514347,691,688,96f42767a45472c8c7ca8c8ce60393fe,REQUEST_ARGS,action,302.0,login2;sa=check;member=1604781
237,3.0,-36.647972,50.59317,1.0,75.296967,17.395275,796,793,03e2b73e19c325b6180c061b075348eb,REQUEST_ARGS,action,200.0,login2;sa=check;member=1599747
344,3.0,-39.238312,51.990406,1.0,75.894005,19.323399,118,118,ade49f489f19268c6a86fd3297a44a85,REQUEST_ARGS,action,200.0,login2;sa=check;member=1629366
359,3.0,-38.862469,51.478909,1.0,75.923775,19.022875,150,150,2efb5d6513b9531915e5c0a24132c91f,REQUEST_ARGS,action,302.0,login2;sa=check;member=1622503
562,3.0,-39.091755,51.562695,1.0,75.934685,19.201685,862,861,b946d896a6c9c33be041a2ece2e603cb,REQUEST_ARGS,action,200.0,login2;sa=check;member=1626280
571,3.0,-38.798176,52.036266,1.0,75.708115,19.152332,882,881,3b81a8323621c88f0d8b3779cefaee8f,REQUEST_ARGS,action,302.0,login2;sa=check;member=1629407
699,3.0,-37.399544,49.642826,1.0,76.066032,17.183109,273,273,6ffa9ccdab7631dbf2e454fa0ceddadc,REQUEST_ARGS,action,302.0,login2;sa=check;member=1556561
758,3.0,-38.011856,51.432167,1.0,75.546333,18.369911,472,472,2ecb2b45f94012c07633ee5b28734410,REQUEST_ARGS,action,200.0,login2;sa=check;member=1499316
837,3.0,-39.473637,50.653698,1.0,76.676575,18.678913,729,729,15da171f06b10d72b879e770bf0f874a,REQUEST_ARGS,action,200.0,login2;sa=check;member=1616993


### Запросы в 5 кластере явно содержат sql-инъекции

In [103]:
# 5 кластер
res_df.loc[res_df[0] == 4]

,0,1,2,3,4,5,6,7,8,9,10,11,12
89,4.0,-43.959904,-6.908885,1.0,36.7281,-27.11548,290,289,7ff8df2a60e32fadefe482bd8378e54d,REQUEST_GET_ARGS,forgot_password,200.0,1AND3527=SELECT3527FROMPGSLEEP5
124,4.0,-45.63356,-7.632851,1.0,38.329235,-28.050463,430,427,08231d6baf4535e8698b22a4c382a41d,REQUEST_HEADERS,Origin,200.0,2yLwd1fN';selectpgsleep2;
232,4.0,-45.800392,-5.47994,1.0,36.832325,-29.030071,783,780,0d70c2c22e0406f5fa8f8ac64c0dee2a,REQUEST_HEADERS,X-Forwarded-Host,200.0,ySdurepu';selectpgsleep4;
264,4.0,-45.771431,-5.100762,1.0,36.639988,-29.221556,886,883,f4c4f58186cc629860198f7cc3205b81,REQUEST_HEADERS,Client-IP,200.0,ZN1lQ9qI';selectpgsleep4;
348,4.0,-46.461182,-6.876787,1.0,38.161266,-28.602592,126,126,9179cbaaea5f8e39f5a78192f9873477,REQUEST_HEADERS,Referer,200.0,H6GcTBRe';selectpgsleep2;
490,4.0,-46.214062,-4.862783,1.0,36.715702,-29.232321,641,640,d81a923c7a384413b560e00c1c3d2c3b,REQUEST_HEADERS,User-Agent,200.0,ekQ6kCAf';selectpgsleep4;
642,4.0,-45.413845,-7.935355,1.0,38.475994,-27.978148,84,84,b677f9039cfa3c204b809cd7e0cd3b5c,REQUEST_HEADERS,Origin,200.0,ZlYLYYvS';selectpgsleep2;
851,4.0,-45.990398,-5.277879,1.0,36.784206,-29.220232,793,793,3cba35bba08890d82d7b87ba4b4f133a,REQUEST_HEADERS,Referer,302.0,ygnhVgAt';selectpgsleep4;
869,4.0,-45.629417,-7.032708,1.0,38.142769,-28.461306,848,846,e1fab37054f452e8a0f5d85139e72020,REQUEST_HEADERS,Origin,200.0,PIdDRBpK';selectpgsleep2;
892,4.0,-47.361897,-5.517334,1.0,37.660587,-30.009836,910,908,e79e3570e619b35d2aa67f779222e937,REQUEST_POST_ARGS,dtid,200.0,fDhMm1XC';selectpgsleep6;


### Насколько могу судить, в 6 кластере содержатся аргументы http-запросов или же метаданные.

In [104]:
# 6 кластер
res_df.loc[res_df[0] == 5]

,0,1,2,3,4,5,6,7,8,9,10,11,12
13,5.0,-23.646915,-33.518295,1.0,45.616364,-17.516645,42,42,f43431c12a9923f44d0525e6684a0a98,REQUEST_ARGS,r,200.0,r42016092711:24:02a:4:i:0;i:1;i:1;i:2;i:2;i:6;...
63,5.0,-23.936243,-33.995289,1.0,44.791683,-17.627323,202,201,5b33aaa6f3413f613270cdc3d845955d,REQUEST_ARGS,r,200.0,r42016092709:36:35a:4:i:0;i:1;i:1;i:2;i:2;i:5;...
100,5.0,-25.128317,-35.47028,1.0,44.563305,-18.929258,339,336,4d124acac9cc397ad38fe68c3a001471,REQUEST_ARGS,r,200.0,r42016092708:34:36a:4:i:0;i:0;i:1;i:3;i:2;i:4;...
439,5.0,-23.38969,-34.130081,1.0,45.675163,-18.17841,445,445,f3727ae2c207de55fc9d7ada6cb791e1,REQUEST_ARGS,r,200.0,r42016092709:07:58a:4:i:0;i:1;i:1;i:3;i:2;i:4;...
480,5.0,-20.678507,-28.623577,1.0,12.987805,-12.53977,607,606,792cd5bb6f9ea2e4b33ef7f7421bbbc0,REQUEST_COOKIES,REQUEST_COOKIES.tl_7765_7770_116,200.0,"a:2:s:8:""referrer"";s:13:""wwwgooglehu"";s:6:""log..."
568,5.0,-23.600243,-33.540897,1.0,45.626438,-17.566046,876,875,e4c4c15512cd6040ced3d7a886270616,REQUEST_ARGS,r,200.0,r42016092709:13:07a:4:i:0;i:1;i:1;i:2;i:2;i:6;...
582,5.0,-24.106142,-34.345913,1.0,45.240845,-17.713438,923,922,b24b312d0001101b63d0a5872c51d409,REQUEST_ARGS,r,200.0,r42016092709:07:28a:4:i:0;i:0;i:1;i:3;i:2;i:5;...
621,5.0,-23.974846,-33.811657,1.0,45.557861,-17.568367,27,27,1433b5ff18ca12ab354023690d2eed98,REQUEST_ARGS,r,200.0,r42016092709:41:06a:4:i:0;i:1;i:1;i:3;i:2;i:6;...
744,5.0,-24.285309,-33.563297,1.0,44.703022,-17.69002,433,433,273a09ba6a4353dfd0ed17c61ee326d1,REQUEST_ARGS,r,200.0,r42016092711:19:51a:4:i:0;i:1;i:1;i:2;i:2;i:5;...
813,5.0,-23.081232,-34.307381,1.0,45.652817,-18.393898,627,627,dda3a4aa64ed761dda803cdf8492d916,REQUEST_ARGS,r,200.0,r42016092408:44:16a:4:i:0;i:1;i:1;i:2;i:2;i:4;...


### Содержимое кластера с "шумом", действительно содержит невалидные значения, с малым количеством осмысленных запросов

In [105]:
# Шум
res_df.loc[res_df[0] == -1]

,0,1,2,3,4,5,6,7,8,9,10,11,12
1,-1.0,8.172576,27.108309,1.0,2.871564,-34.436966,5,5,d3743773c596f478093bcc8c2933db65,REQUEST_ARGS_KEYS,M}PÂ¶WMÂ¤_Â§ÃÃ <Ã¬?{#Â8Ã»ÎA?tÂ5ÂÂ©Ø½...,200.0,MPWM8At5
2,-1.0,55.249565,5.078446,1.0,-4.430802,39.482944,6,6,a43d37066673e80c8b59826f731fbc4c,REQUEST_ARGS,image,200.0,/imgbasa/17896173683jpg
4,-1.0,32.567371,-15.952653,1.0,17.04377,10.223716,15,15,b7caea0cc6d15508ff30f95f3e599160,RESPONSE_HEADERS,RESPONSE_HEADERS.Age,200.0,1443
5,-1.0,22.327614,-1.550468,1.0,15.126709,30.895777,16,16,3b1b8a47a4ead29d94f985f54c874795,RESPONSE_HEADERS,RESPONSE_HEADERS.AN-X-Request-Uuid,200.0,e9c39ad02ad449788cf1d704aad75b52
7,-1.0,-26.288771,-47.559044,1.0,57.354885,3.030128,27,27,193b27931dc601ac9f525ec9cc18a7d1,REQUEST_GET_ARGS,REQUEST_GET_ARGS.time0,200.0,01:43
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3290,-1.0,20.256632,-32.255291,1.0,35.112309,17.490442,968,961,693d154af00339f7fbbc0fd36fca2f76,REQUEST_JSON,REQUEST_JSON.cts,200.0,1516780253
3293,-1.0,-46.707546,13.771709,1.0,19.964436,-60.736725,977,970,80f3799a0d88a98d08bff6fe071553ab,REQUEST_ARGS_KEYS,Â¾Â_Ä±-ÂÂ#ÂzÃbÂÏ°lÃ´Ã®Ã$ÃµÃÃ¸Â¦ÃÃ...,200.0,zblPmhryvX
3295,-1.0,11.903955,-7.843584,1.0,19.634377,32.894646,979,972,3b4e02fb1c25672db73de5ad82b2712b,REQUEST_JSON,REQUEST_JSON.answerId,200.0,e7c3c39cef464d38a095e1ef60671b66
3297,-1.0,-17.501911,-20.022917,1.0,7.534424,-6.372557,984,977,f6240dc3bfdce2f625bbb5ff09a3327e,REQUEST_POST_ARGS,PaymentSum,200.0,'responsewrite92692879186684'
